## 20200603 Wednesday

In [1]:
from torchtext.data import Field, BucketIterator, interleave_keys
from torchtext.datasets import TranslationDataset
import mosestokenizer
import torch
from typing import Tuple
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import Tensor
import math
import time

# Author: WonKee Lee (POSTECH)
# "Neural Machine Translation by Jointly Learning to Align and Translate" 논문의 model 재현 (Toy code)
#  (https://pytorch.org/tutorials/beginner/torchtext_translation_tutorial.html 를 참고하여 수정함.)

In [2]:
### torchtext #####

In [3]:
BOS = '<s>'
EOS = '</s>'
PAD = '<pad>'

tokenizer_en = mosestokenizer.MosesTokenizer('en')
tokenizer_de = mosestokenizer.MosesTokenizer('de')

# Field: Tensor로 표현할 데이터의 타입, 처리 프로세스 등을 정의하는 객체
src = Field(sequential=True,
            use_vocab=True,
            pad_token=PAD,
            tokenize=tokenizer_en, # 어떤식으로 tokenizer 할건지 결정
            lower=True, # 소문자 처리
            batch_first=True) # [Batch, length] if False [length, Batch]

tgt = Field(sequential=True,
            use_vocab=True,
            pad_token=PAD,
            tokenize=tokenizer_de,
            lower=True,
            init_token=BOS,
            eos_token=EOS,
            batch_first=True)

In [4]:
prefix_f = './escape.en-de.tok.5k' # 5k는 데이터 사이즈 의미

# parallel data 각각 (en, de) 을 src Field 와 tgt Field에 정의된 형태로 처리.
parallel_dataset = TranslationDataset(path=prefix_f, exts=('.en', '.de'), fields=[('src', src), ('tgt', tgt)])


In [5]:
print(parallel_dataset) 

print(parallel_dataset.examples[0].__dict__.items()) # src 및 tgt 에 대한 samples 를 포함.

dict_items([('src', ['once', 'again', ',', 'we', 'are', 'ignoring', 'the', 'fundamental', 'problem', ',', 'which', 'is', 'that', 'as', 'long', 'as', 'there', 'is', 'no', 'democracy', 'in', 'belgrade', ',', 'there', 'will', 'be', 'no', 'solution', 'for', 'kosovo', ',', 'just', 'as', 'there', 'will', 'be', 'no', 'solution', 'for', 'the', 'entire', 'yugoslav', 'population', '.']), ('tgt', ['das', 'bedeutet', ',', 'einmal', 'mehr', 'die', 'augen', 'vor', 'dem', 'grundproblem', 'zu', 'verschließen', ',', 'daß', 'es', 'nämlich', ',', 'solange', 'es', 'keine', 'demokratie', 'in', 'belgrad', 'gibt', ',', 'auch', 'keine', 'lösung', 'für', 'den', 'kosovo', 'geben', 'wird', ',', 'ebenso', 'wenig', 'für', 'die', 'gesamte', 'bevölkerung', 'jugoslawiens', '.', 'das', 'muß', 'man', 'endlich', 'begreifen', '.'])])


In [6]:
print(parallel_dataset.examples[0].src) # 첫번째 src 문장

['once', 'again', ',', 'we', 'are', 'ignoring', 'the', 'fundamental', 'problem', ',', 'which', 'is', 'that', 'as', 'long', 'as', 'there', 'is', 'no', 'democracy', 'in', 'belgrade', ',', 'there', 'will', 'be', 'no', 'solution', 'for', 'kosovo', ',', 'just', 'as', 'there', 'will', 'be', 'no', 'solution', 'for', 'the', 'entire', 'yugoslav', 'population', '.']


In [7]:
print(parallel_dataset.examples[0].tgt) # 첫번째 tgt 문장

['das', 'bedeutet', ',', 'einmal', 'mehr', 'die', 'augen', 'vor', 'dem', 'grundproblem', 'zu', 'verschließen', ',', 'daß', 'es', 'nämlich', ',', 'solange', 'es', 'keine', 'demokratie', 'in', 'belgrad', 'gibt', ',', 'auch', 'keine', 'lösung', 'für', 'den', 'kosovo', 'geben', 'wird', ',', 'ebenso', 'wenig', 'für', 'die', 'gesamte', 'bevölkerung', 'jugoslawiens', '.', 'das', 'muß', 'man', 'endlich', 'begreifen', '.']


In [8]:
##### 사전 구축 ########
# src, tgt 필드에 사전 구축
src.build_vocab(parallel_dataset, min_freq=5, max_size=15000)
tgt.build_vocab(parallel_dataset, min_freq=5, max_size=15000)

In [9]:
# 사전 내용 
print(src.vocab.__dict__.keys())
print('')
for i, (k, v) in enumerate(src.vocab.stoi.items()): # stoi: word와 index 페어 정보를 얻을 수 있음
    print ('{:>10s} | {:>3d}'.format(k, v)) 
    if i == 15 : break

dict_keys(['freqs', 'itos', 'unk_index', 'stoi', 'vectors'])

     <unk> |   0
     <pad> |   1
       the |   2
         , |   3
         . |   4
        of |   5
        to |   6
       and |   7
        in |   8
         a |   9
       @-@ |  10
        is |  11
       for |  12
         ) |  13
         ( |  14
      that |  15


In [10]:
train, valid = parallel_dataset.split(split_ratio=0.95) # 0.95 = train / 0.05 = valid 데이터로 분할

In [11]:
# Batch iterator 생성.
# iterator 를 반복할 때 마다, batch 크기의 (src, tgt) 쌍의 parallel data가 출력됨.
BATCH_SIZE = 3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator = BucketIterator.splits((train, valid), batch_size=BATCH_SIZE, # 각각에 대한 iterator를 뽑아준다.
                                                    sort_key=lambda x: interleave_keys(len(x.src), len(x.tgt)), # 정렬의 방식 정의 (src 와 tgt를 둘다 고려하여 패딩을 최소화)
                                                    device=device)

In [12]:
# iterator 는 Batch 객체를 출력해주며, 
# Batch.src / Batch.tgt 로 parallel data각각에 대해 접근가능.

# 예시.
Batch = next(iter(train_iterator)) 

In [13]:
# src 에 저장된 데이터 출력
# Field에 정의된 형식으로 데이터 전처리 (indexing 포함.)
# 가장 긴 문장을 기준으로, 그 보다 짧은 문장은 Padding idx(=1) 을 부여.
Batch.src 

tensor([[  23,   11,   73, 1634,   15,   32,    0,    0,  188,   68,  143,    8,
            2,    0,  422,    4,    1,    1,    1,    1,    1,    1,    1,    1,
            1],
        [  27,   49, 2261,   15,   23,   55,   37,    2, 1308, 2293,    5,  218,
          133, 2749,    7,    0,    2, 2508,   19,    0,    6,    0,    0,  113,
            4],
        [  52,   33,    6,   26,   62,    3,   87,    8,   15,    2, 2841,    0,
           11,    0,   24,    2,  751,  193,   10,   62,    4,    1,    1,    1,
            1]], device='cuda:0')

In [14]:
# Field에 정의된 형식으로 데이터 전처리 (indexing + bos + eos 토큰 처리 됨.)
Batch.tgt 

tensor([[   2,   31,   15,  366,    0,    4,   57,   44,    0,    4,   57,  264,
            0,    0, 1199,  269,   41,    5,    3,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1],
        [   2,   31,   15,    0,    4,   35,   31,   34,    6,    0, 1631, 1768,
          146,    4,    6,   17,    0,   26,    0,    0,    8,    0,   62,    4,
            0,  159,    0,    5,    3],
        [   2,   77,   18,   30,   75,    4,   69,   70,    4,   35,   37,    7,
            0, 1208,   45,   29,    0,  174,    9,   75,  277,    5,    3,    1,
            1,    1,    1,    1,    1]], device='cuda:0')

In [15]:
###############################################################################
######                      Network 정의                               ########
###############################################################################

In [16]:
### Encoder 정의.

class Encoder(nn.Module):
    def __init__(self, hidden_dim: int, src_ntoken: int, dropout: float):
        super().__init__()

        self.hidden_dim = hidden_dim
        self.dropout = dropout
        self.src_ntoken = src_ntoken

        self.embedding = nn.Embedding(src_ntoken, hidden_dim, ## 사전의 크기
                                      padding_idx=src.vocab.stoi['<pad>'])
        self.rnn = nn.GRU(hidden_dim, hidden_dim, # first hidden_dim: 입력, second: 출력
                          bidirectional = True, batch_first=True) # batch_first = [B, L, dim]
        
        
        self.fc = nn.Linear(hidden_dim * 2, hidden_dim) # bidirectional hidden을 하나의 hidden size로 변환
        self.dropout = nn.Dropout(dropout)

    def forward(self, src: Tensor) -> Tuple[Tensor]:

        embedded = self.dropout(self.embedding(src)) # src idx 들이 embedding으로 바뀐다.
                                                     # src = (Batch, Length), shape = (Batch, Length, hidden_dim)
        # outputs: [B, L, D*2], hidden: [2, B, D] D = dimention, 2 = bidirectional 을 의미
        # Note: if bidirectional=False then [B, L, D], [1, B, D]
        outputs, hidden = self.rnn(embedded)

        last_hidden = self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)) # [B, D]
        hidden = torch.tanh(last_hidden) # last bidirectional hidden

        return outputs, hidden

In [17]:
### Attention 모듈 정의 ###

class Attention(nn.Module):
    def __init__(self,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 attn_dim: int):
        super().__init__()

        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim

        attn_in = (enc_hid_dim * 2) + dec_hid_dim # bidirectional hidden + dec_hidden
        self.linear = nn.Linear(attn_in, attn_dim)
        self.merge = nn.Linear(attn_dim, 1)

    def forward(self, decoder_hidden: Tensor, encoder_outputs: Tensor) -> Tensor:
        # decoder_hidden = (Batch, 1, Dim)
        src_len = encoder_outputs.shape[1]
        repeated_decoder_hidden = decoder_hidden.repeat(1, src_len, 1) # [B, src_len, D]

        # enc의 각 step의 hidden + decoder의 hidden 의 결과값 # [B, src_len, D*2] --> [B, src_len, D]
        # tanh(W*h_dec  + U*h_enc) 수식 부분.
        energy = torch.tanh(self.linear(torch.cat((
            repeated_decoder_hidden,
            encoder_outputs),
            dim = 2))) 

        score = self.merge(energy).squeeze(-1) # [B, src_len] 각 src 단어에 대한 점수 -> V^T tanh(W*h_dec  + U*h_enc) 부분
        normalized_score = F.softmax(score, dim=1)  # softmax를 통해 확률분포값으로 변환
        return  normalized_score

In [18]:
### Decoder 모듈 정의 ####

class Decoder(nn.Module):
    def __init__(self, hidden_dim: int, dec_ntoken: int, dropout: int):
        super().__init__()

        self.hidden_dim = hidden_dim
        self.dropout = dropout
        self.attention = Attention(enc_hid_dim=hidden_dim,
                                   dec_hid_dim=hidden_dim,
                                   attn_dim=hidden_dim) # attn module, init 실행
        
        
        self.dec_ntoken = dec_ntoken # vocab_size

        self.embedding = nn.Embedding(dec_ntoken, hidden_dim, padding_idx=tgt.vocab.stoi['<pad>']) # bidirection이 아님
        self.rnn = nn.GRU(hidden_dim, hidden_dim, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(self.hidden_dim*3, dec_ntoken)
        self.sm = nn.LogSoftmax(dim=-1) # softmax activation

    def _context_rep(self, dec_out: Tensor, enc_outs: Tensor) -> Tensor:

        scores = self.attention(dec_out, enc_outs) # [B, L]
        scores = scores.unsqueeze(1) # [B, 1, src_len] -> weight value (softmax)

        # scores: (batch, 1, src_len),  ecn_outs: (Batch, src_len, dim)
        context_vector = torch.bmm(scores, enc_outs) # weighted average -> (batch, 1, dec_dim): encoder의 각 hidden의 weighted sum
        return context_vector

    def forward(self, input: Tensor, decoder_hidden: Tensor, encoder_outputs: Tensor) -> Tuple[Tensor]:

        dec_outs = []

        embedded = self.dropout(self.embedding(input)) # (Batch, length, Dim)
        decoder_hidden = decoder_hidden.unsqueeze(0) # hidden = (Batch, Dim) --> (1, Batch, Dim)
        # (Batch,1, dim) (batch, 1 ,dim)...
        for emb_t in embedded.split(1, dim=1): # Batch의 각 time step (=각 단어) 에 대한 embedding 출력 
            rnn_out, decoder_hidden = self.rnn(emb_t, decoder_hidden) # feed input with previous decoder hidden at each step

            context = self._context_rep(rnn_out, encoder_outputs)
            rnn_context = self.dropout(torch.cat([rnn_out, context], dim=2))
            dec_out = self.linear(rnn_context)
            dec_outs += [self.sm(dec_out)]

        dec_outs = dec_outs[:-1] # trg = trg[:-1] # <E> 는 Decoder 입력으로 고려하지 않음.
        dec_outs = torch.cat(dec_outs, dim=1) # convert list to tensor: [Batch, Length, Vocab_size]
        return dec_outs

In [19]:
### Seq-to-Seq 모델 정의 ###

class Seq2Seq(nn.Module):
    def __init__(self,
                 encoder: nn.Module,
                 decoder: nn.Module,
                 device: torch.device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src: Tensor, trg: Tensor) -> Tensor:
        encoder_outputs, hidden = self.encoder(src) # encoder_outputs = (Batch, length, Dim *2), hidden = (Batch, Dim)
        dec_out = self.decoder(trg, hidden, encoder_outputs)
        return dec_out


In [20]:
INPUT_DIM = len(src.vocab)  # src 사전 크기
OUTPUT_DIM = len(tgt.vocab) # tgt 사전 크기
HID_DIM = 64 # rnn, embedding, 등. 모든 hidden 크기를 해당 값으로 통일함. (실습의 용이성을 위함.)
D_OUT = 0.3 # Dropout  확률

# 상단의 예시에서 작은 Batch로 보여줬기 때문에, 
# Batch 크기를 원래대로 바꿔 iterator 다시 선언함.
BATCH_SIZE = 15
train_iterator, valid_iterator = BucketIterator.splits((train, valid), batch_size=BATCH_SIZE,
                                                    sort_key=lambda x: interleave_keys(len(x.src), len(x.tgt)),
                                                    device=device)

In [21]:
# 인코더 및 디코더 생성
# Seq2Seq 모델 생성
encoder = Encoder(HID_DIM, INPUT_DIM, D_OUT)
decoder = Decoder(HID_DIM, OUTPUT_DIM, D_OUT)
model = Seq2Seq(encoder, decoder, device).to(device)

In [22]:
def init_weights(m: nn.Module):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)


model.apply(init_weights) # 모델 파라미터 초기화
optimizer = optim.Adam(model.parameters()) # Optimizer 설정
criterion = nn.CrossEntropyLoss(ignore_index=tgt.vocab.stoi['<pad>']) # LOSS 설정

In [23]:
# 모델 정보 및 파라미터 수 출력
def count_parameters(model: nn.Module):
    print(model)
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(2893, 64, padding_idx=1)
    (rnn): GRU(64, 64, batch_first=True, bidirectional=True)
    (fc): Linear(in_features=128, out_features=64, bias=True)
    (dropout): Dropout(p=0.3, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (linear): Linear(in_features=192, out_features=64, bias=True)
      (merge): Linear(in_features=64, out_features=1, bias=True)
    )
    (embedding): Embedding(2384, 64, padding_idx=1)
    (rnn): GRU(64, 64, batch_first=True)
    (dropout): Dropout(p=0.3, inplace=False)
    (linear): Linear(in_features=192, out_features=2384, bias=True)
    (sm): LogSoftmax()
  )
)
The model has 893,393 trainable parameters


In [24]:
## 모델 학습 함수 ###
def train(model: nn.Module, iterator: BucketIterator,
          optimizer: optim.Optimizer, criterion: nn.Module, clip: float):

    model.train()
    epoch_loss = 0

    for i, batch in enumerate(iterator):
        src = batch.src
        tgt = batch.tgt

        optimizer.zero_grad() # gradient 제거

        output = model(src, tgt) # [batch, length, vocab_size]
        output = output.view(-1, output.size(-1)) # flatten --> (batch * length, vocab_size)

        tgt = tgt.unsqueeze(-1)[:,1:,:].squeeze(-1).contiguous() # remove <S> placed at first from targets
        tgt = tgt.view(-1) # flatten target with shape = (batch * length)

        loss = criterion(output, tgt) # tgt 이 내부적으로 one_hot으로 변환됨 --> (batch * length, vocab_size)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()
        epoch_loss += loss.item()

        if(((i+1) % int(len(iterator)*0.2)) == 0):
            num_complete = batch.batch_size * (i+1)
            total_size = batch.batch_size * int(len(iterator))
            ratio = num_complete/total_size * 100
            print('| Current Epoch:  {:>4d} / {:<5d} ({:2d}%) | Train Loss: {:3.3f}'.
                  format(num_complete, batch.batch_size * int(len(iterator)), round(ratio), loss.item())
                  )

    return epoch_loss / len(iterator)

In [25]:
### 모델 평가 함수 ###
def evaluate(model: nn.Module, iterator: BucketIterator,
             criterion: nn.Module):
    model.eval()
    epoch_loss = 0

    with torch.no_grad():
        for _, batch in enumerate(iterator):
            src = batch.src
            tgt = batch.tgt

            output = model(src, tgt)
            output = output.view(-1, output.size(-1)) # flatten (batch * length, vocab_size)

            tgt = tgt.unsqueeze(-1)[:,1:,:].squeeze(-1).contiguous() # remove <S> placed at first from targets
            tgt = tgt.view(-1) # flatten target with shape = (batch * length)
            loss = criterion(output, tgt)

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [26]:
# 학습 시간 카운트를 위한 기타 함수 #
def epoch_time(start_time: int, end_time: int):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [27]:
N_EPOCHS = 10 # 최대 epoch 크기
CLIP = 0.5 # weight cliping 

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    print('='*65)
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
    print('='*65)

| Current Epoch:   945 / 4755  (20%) | Train Loss: 5.062
| Current Epoch:  1890 / 4755  (40%) | Train Loss: 5.118
| Current Epoch:  2835 / 4755  (60%) | Train Loss: 4.605
| Current Epoch:  3780 / 4755  (79%) | Train Loss: 4.658
| Current Epoch:  4725 / 4755  (99%) | Train Loss: 4.748
Epoch: 01 | Time: 0m 55s
	Train Loss: 5.174 | Train PPL: 176.572
	 Val. Loss: 4.825 |  Val. PPL: 124.567
| Current Epoch:   945 / 4755  (20%) | Train Loss: 4.596
| Current Epoch:  1890 / 4755  (40%) | Train Loss: 4.642
| Current Epoch:  2835 / 4755  (60%) | Train Loss: 4.904
| Current Epoch:  3780 / 4755  (79%) | Train Loss: 4.492
| Current Epoch:  4725 / 4755  (99%) | Train Loss: 4.278
Epoch: 02 | Time: 0m 55s
	Train Loss: 4.653 | Train PPL: 104.884
	 Val. Loss: 4.513 |  Val. PPL:  91.235
| Current Epoch:   945 / 4755  (20%) | Train Loss: 4.228
| Current Epoch:  1890 / 4755  (40%) | Train Loss: 4.183
| Current Epoch:  2835 / 4755  (60%) | Train Loss: 4.618
| Current Epoch:  3780 / 4755  (79%) | Train Loss